In [ ]:
import os
os.chdir('..')
from dataset import *

In [ ]:
import logging
from torchvision.transforms import ToPILImage
from torchvision.utils import make_grid
from IPython.display import display
from torch.utils.data import ConcatDataset
charset = CharsetMapper('data/charset_36.txt')

In [ ]:
def show_all(dl, iter_size=None):
    if iter_size is None: iter_size = len(dl)
    for i, item in enumerate(dl):
        if i >= iter_size:
             break
        image = item[0]
        label = item[1][0]
        length = item[1][1]
        print(f'iter {i}:', [charset.get_text(label[j][0: length[j]].argmax(-1), padding=False) for j in range(bs)])
        display(ToPILImage()(make_grid(item[0].cpu())))

# Construct dataset

In [ ]:
data1 = ImageDataset('data/training/ST', is_training=True);data1  # is_training

In [ ]:
bs=64
data2 = ImageDataBunch.create(train_ds=data1, valid_ds=None, bs=bs, num_workers=1);data2

In [ ]:
#data3 = data2.normalize(imagenet_stats);data3
data3 = data2

In [ ]:
show_all(data3.train_dl, 4)

# Add dataset

In [ ]:
kwargs = {'data_aug': False, 'is_training': False}

In [ ]:
data1 = ImageDataset('data/evaluation/IIIT5k_3000', **kwargs);data1

In [ ]:
data2 = ImageDataset('data/evaluation/SVT', **kwargs);data2

In [ ]:
data3 = ConcatDataset([data1, data2])

In [ ]:
bs=64
data4 = ImageDataBunch.create(train_ds=data1, valid_ds=data3, bs=bs, num_workers=1);data4

In [ ]:
len(data4.train_dl), len(data4.valid_dl)

In [ ]:
show_all(data4.train_dl, 4)

# TEST

In [ ]:
len(data4.valid_dl)

In [ ]:
import time
niter = 1000
start = time.time()
for i, item in enumerate(progress_bar(data4.valid_dl)):
    if i % niter == 0 and i > 0:
        print(i, (time.time() - start) / niter)
        start = time.time()

In [ ]:
num = 20
index = 6
plt.figure(figsize=(20, 10))
for i in range(num):
    plt.subplot(num // 4, 4, i+1)
    plt.imshow(data4.train_ds[i][0].data.numpy().transpose(1,2,0))

In [ ]:
def show(path, image_key):
    with lmdb.open(str(path), readonly=True, lock=False, readahead=False, meminit=False).begin(write=False) as txn:
        imgbuf = txn.get(image_key.encode())  # image
        buf = six.BytesIO()
        buf.write(imgbuf)
        buf.seek(0)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
            x = PIL.Image.open(buf).convert('RGB')
        print(x.size)
        plt.imshow(x)

In [ ]:
image_key = 'image-003118258'
image_key = 'image-002780217'
image_key = 'image-002780218'
path = 'data/CVPR2016'
show(path, image_key)

In [ ]:
image_key = 'image-004668347'
image_key = 'image-006128516'
path = 'data/NIPS2014'
show(path, image_key)

In [ ]:
image_key = 'image-004668347'
image_key = 'image-000002420'
path = 'data/IIIT5K_3000'
show(path, image_key)